In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from public_data.models import CouvertureSol, UsageSol, ArtifCommune
from project.models import Project
from prettytable import PrettyTable

In [3]:
project = Project.objects.get(pk=4)
project

<Project: COBAS_SHAPE>

In [4]:
headers = ['nom', ]
headers += ArtifCommune.list_attr()

table = PrettyTable(field_names=headers, align="l")

for city in project.cities.all():
    row = [city.name, ]
    row += city.list_artif()
    table.add_row(row)

print(table)

+--------------------+-------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
| nom                | artif_before_2009 | artif_2009 | artif_2010 | artif_2011 | artif_2012 | artif_2013 | artif_2014 | artif_2015 | artif_2016 | artif_2017 | artif_2018 |
+--------------------+-------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
| ANDERNOS-LES-BAINS | 740.65            | 7.04       | 5.45       | 14.71      | 5.40       | 4.20       | 6.71       | 5.58       | 6.51       | 6.33       | 4.36       |
| AUDENGE            | 518.38            | 4.00       | 5.62       | 5.83       | 5.38       | 4.55       | 6.90       | 7.65       | 7.95       | 7.75       | 5.45       |
| BIGANOS            | 644.37            | 9.97       | 8.62       | 13.43      | 12.69      | 6.35       | 9.41       | 6.27       | 8

In [5]:
# initialise le DataFrame avec les lignes
indexes = ArtifCommune.list_attr()
df = pd.DataFrame(0, index=indexes, columns=list())
df

""
artif_before_2009
artif_2009
artif_2010
artif_2011
artif_2012
artif_2013
artif_2014
artif_2015
artif_2016
artif_2017


In [6]:
# on rempli le tableau avec les villes du projet
for city in project.cities.all():
    df[city.name] = city.list_artif()
df

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
artif_before_2009,740.65,518.38,644.37,238.64,643.12,394.51,488.49,1250.64
artif_2009,7.04,4.00,9.97,3.86,11.76,4.74,3.81,13.29
artif_2010,5.45,5.62,8.62,4.88,12.00,7.42,4.01,8.01
artif_2011,14.71,5.83,13.43,3.02,15.04,3.86,5.12,8.28
artif_2012,5.40,5.38,12.69,2.66,15.59,4.07,6.09,6.56
artif_2013,4.20,4.55,6.35,0.93,17.28,5.11,4.76,11.78
artif_2014,6.71,6.90,9.41,1.69,12.21,10.85,3.36,10.95
artif_2015,5.58,7.65,6.27,2.76,9.97,3.00,1.96,11.70
artif_2016,6.51,7.95,8.73,0.92,12.68,6.39,2.90,10.00
artif_2017,6.33,7.75,4.60,3.73,18.23,2.53,1.96,7.85


In [7]:
df.T

,artif_before_2009,artif_2009,artif_2010,artif_2011,artif_2012,artif_2013,artif_2014,artif_2015,artif_2016,artif_2017,artif_2018
ANDERNOS-LES-BAINS,740.65,7.04,5.45,14.71,5.40,4.20,6.71,5.58,6.51,6.33,4.36
AUDENGE,518.38,4.00,5.62,5.83,5.38,4.55,6.90,7.65,7.95,7.75,5.45
BIGANOS,644.37,9.97,8.62,13.43,12.69,6.35,9.41,6.27,8.73,4.60,3.26
MARCHEPRIME,238.64,3.86,4.88,3.02,2.66,0.93,1.69,2.76,0.92,3.73,1.90
MIOS,643.12,11.76,12.00,15.04,15.59,17.28,12.21,9.97,12.68,18.23,13.93
ARES,394.51,4.74,7.42,3.86,4.07,5.11,10.85,3.00,6.39,2.53,2.03
LANTON,488.49,3.81,4.01,5.12,6.09,4.76,3.36,1.96,2.90,1.96,2.67
LEGE-CAP-FERRET,1250.64,13.29,8.01,8.28,6.56,11.78,10.95,11.70,10.00,7.85,5.91


In [8]:
# on réduit le DF aux données qui nous intéressent
year_min = 2014
year_max = 2016


index_to_keep = index_to_sum = index_to_exclude = None

# identification des années supérieures qui ne nous intéresse pas (to exclude)
index_to_exclude = []
for i in indexes:
    year = int(i.split("_")[-1])
    if year > year_max:
        index_to_exclude.append(i)
        
index_to_exclude

['artif_2017', 'artif_2018']

In [9]:
# on identifie les index qui font parties des années à garder
index_to_keep = {f"artif_{y}" for y in range(year_min, year_max + 1)}
index_to_keep = set(indexes).intersection(index_to_keep)
index_to_keep

{'artif_2014', 'artif_2015', 'artif_2016'}

In [10]:
# on déduit les années à exclure qu'il va falloir sommer 
index_to_sum = set(indexes) - index_to_keep - set(index_to_exclude)
index_to_sum

{'artif_2009',
 'artif_2010',
 'artif_2011',
 'artif_2012',
 'artif_2013',
 'artif_before_2009'}

In [11]:
# on construit le df des années que l'on va devoir sommer
df_sum = df.loc[list(index_to_sum)]
df_sum

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
artif_2013,4.20,4.55,6.35,0.93,17.28,5.11,4.76,11.78
artif_2009,7.04,4.00,9.97,3.86,11.76,4.74,3.81,13.29
artif_2011,14.71,5.83,13.43,3.02,15.04,3.86,5.12,8.28
artif_before_2009,740.65,518.38,644.37,238.64,643.12,394.51,488.49,1250.64
artif_2012,5.40,5.38,12.69,2.66,15.59,4.07,6.09,6.56
artif_2010,5.45,5.62,8.62,4.88,12.00,7.42,4.01,8.01


In [12]:
df_sum.sum()

ANDERNOS-LES-BAINS     777.45
AUDENGE                543.76
BIGANOS                695.43
MARCHEPRIME            253.99
MIOS                   714.79
ARES                   419.71
LANTON                 512.28
LEGE-CAP-FERRET       1298.56
dtype: object

In [13]:
avant = total = None
df_final = df.loc[index_to_keep]
df_final

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
artif_2016,6.51,7.95,8.73,0.92,12.68,6.39,2.90,10.00
artif_2014,6.71,6.90,9.41,1.69,12.21,10.85,3.36,10.95
artif_2015,5.58,7.65,6.27,2.76,9.97,3.00,1.96,11.70


In [14]:
# on ajoute la ligne "artificialisée avant"
avant = pd.DataFrame(df_sum.sum(), index=df.columns, columns=[f"Avant {year_min}"])
avant = avant.T
df_final = avant.append(df_final)
df_final

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
Avant 2014,777.45,543.76,695.43,253.99,714.79,419.71,512.28,1298.56
artif_2016,6.51,7.95,8.73,0.92,12.68,6.39,2.90,10.00
artif_2014,6.71,6.90,9.41,1.69,12.21,10.85,3.36,10.95
artif_2015,5.58,7.65,6.27,2.76,9.97,3.00,1.96,11.70


In [15]:
total = pd.DataFrame(df_final.sum(), index=df.columns, columns=["Total"]).T
df_final = df_final.append(total)
df_final.T

,Avant 2014,artif_2016,artif_2014,artif_2015,Total
ANDERNOS-LES-BAINS,777.45,6.51,6.71,5.58,796.25
AUDENGE,543.76,7.95,6.90,7.65,566.26
BIGANOS,695.43,8.73,9.41,6.27,719.84
MARCHEPRIME,253.99,0.92,1.69,2.76,259.36
MIOS,714.79,12.68,12.21,9.97,749.65
ARES,419.71,6.39,10.85,3.00,439.95
LANTON,512.28,2.90,3.36,1.96,520.50
LEGE-CAP-FERRET,1298.56,10.00,10.95,11.70,1331.21


In [16]:
df2 = df_final.T
df2

,Avant 2014,artif_2016,artif_2014,artif_2015,Total
ANDERNOS-LES-BAINS,777.45,6.51,6.71,5.58,796.25
AUDENGE,543.76,7.95,6.90,7.65,566.26
BIGANOS,695.43,8.73,9.41,6.27,719.84
MARCHEPRIME,253.99,0.92,1.69,2.76,259.36
MIOS,714.79,12.68,12.21,9.97,749.65
ARES,419.71,6.39,10.85,3.00,439.95
LANTON,512.28,2.90,3.36,1.96,520.50
LEGE-CAP-FERRET,1298.56,10.00,10.95,11.70,1331.21


In [17]:
# Add progression column (final - init) / init
progression = (df2.loc[:, "Total"] - df2.loc[:, f"Avant {year_min}"]) / df2.loc[:, f"Avant {year_min}"]
progression

ANDERNOS-LES-BAINS    0.02418161939674577143224644672
AUDENGE               0.04137854936001176989848462557
BIGANOS               0.03510058524941403160634427620
MARCHEPRIME           0.02114256466790031103586755384
MIOS                  0.04876956868450873683179675149
ARES                  0.04822377355793285840223011127
LANTON                0.01604591239166081049426095104
LEGE-CAP-FERRET       0.02514323558403154263183834401
dtype: object

In [18]:
indexes = [f"artif_{y}" for y in range(year_min, year_max + 1)]
indexes = [f"Avant {year_min}"] + indexes + ["Total", "Progression"]
df_final = df_final.reindex(indexes)
df_final

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
Avant 2014,777.45,543.76,695.43,253.99,714.79,419.71,512.28,1298.56
artif_2014,6.71,6.90,9.41,1.69,12.21,10.85,3.36,10.95
artif_2015,5.58,7.65,6.27,2.76,9.97,3.00,1.96,11.70
artif_2016,6.51,7.95,8.73,0.92,12.68,6.39,2.90,10.00
Total,796.25,566.26,719.84,259.36,749.65,439.95,520.50,1331.21
Progression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df3 = df_final.T
df3["Progression"] = progression
df3

,Avant 2014,artif_2014,artif_2015,artif_2016,Total,Progression
ANDERNOS-LES-BAINS,777.45,6.71,5.58,6.51,796.25,0.02418161939674577143224644672
AUDENGE,543.76,6.90,7.65,7.95,566.26,0.04137854936001176989848462557
BIGANOS,695.43,9.41,6.27,8.73,719.84,0.03510058524941403160634427620
MARCHEPRIME,253.99,1.69,2.76,0.92,259.36,0.02114256466790031103586755384
MIOS,714.79,12.21,9.97,12.68,749.65,0.04876956868450873683179675149
ARES,419.71,10.85,3.00,6.39,439.95,0.04822377355793285840223011127
LANTON,512.28,3.36,1.96,2.90,520.50,0.01604591239166081049426095104
LEGE-CAP-FERRET,1298.56,10.95,11.70,10.00,1331.21,0.02514323558403154263183834401


In [20]:
table = []
for city, row in df3.iterrows():
    table.append({
        "name": city,
        "before": row[0],
        "items": list(row[1:-2]),
        "total": row[-2],
        # "surface": city.surface,
        "progression": f"{row[-1]}%",
    })
table

[{'name': 'ANDERNOS-LES-BAINS',
  'before': Decimal('777.45'),
  'items': [Decimal('6.71'), Decimal('5.58'), Decimal('6.51')],
  'total': Decimal('796.25'),
  'progression': '0.02418161939674577143224644672%'},
 {'name': 'AUDENGE',
  'before': Decimal('543.76'),
  'items': [Decimal('6.90'), Decimal('7.65'), Decimal('7.95')],
  'total': Decimal('566.26'),
  'progression': '0.04137854936001176989848462557%'},
 {'name': 'BIGANOS',
  'before': Decimal('695.43'),
  'items': [Decimal('9.41'), Decimal('6.27'), Decimal('8.73')],
  'total': Decimal('719.84'),
  'progression': '0.03510058524941403160634427620%'},
 {'name': 'MARCHEPRIME',
  'before': Decimal('253.99'),
  'items': [Decimal('1.69'), Decimal('2.76'), Decimal('0.92')],
  'total': Decimal('259.36'),
  'progression': '0.02114256466790031103586755384%'},
 {'name': 'MIOS',
  'before': Decimal('714.79'),
  'items': [Decimal('12.21'), Decimal('9.97'), Decimal('12.68')],
  'total': Decimal('749.65'),
  'progression': '0.04876956868450873683

In [21]:
df_final

,ANDERNOS-LES-BAINS,AUDENGE,BIGANOS,MARCHEPRIME,MIOS,ARES,LANTON,LEGE-CAP-FERRET
Avant 2014,777.45,543.76,695.43,253.99,714.79,419.71,512.28,1298.56
artif_2014,6.71,6.90,9.41,1.69,12.21,10.85,3.36,10.95
artif_2015,5.58,7.65,6.27,2.76,9.97,3.00,1.96,11.70
artif_2016,6.51,7.95,8.73,0.92,12.68,6.39,2.90,10.00
Total,796.25,566.26,719.84,259.36,749.65,439.95,520.50,1331.21
Progression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
final = df_final.loc["Total"].sum()
final

Decimal('5383.02')

In [23]:
label = f"Avant {year_min}"
init = df_final.loc[label].sum()
init

Decimal('5215.97')

In [24]:
global_progression = (final - init) / init
global_progression

Decimal('0.03202664125752257010680659590')

In [25]:
df_final.sum()

ANDERNOS-LES-BAINS    1592.50
AUDENGE               1132.52
BIGANOS               1439.68
MARCHEPRIME            518.72
MIOS                  1499.30
ARES                   879.90
LANTON                1041.00
LEGE-CAP-FERRET       2662.42
dtype: object